<a href="https://colab.research.google.com/github/RishiLal/CV-FirstHtmlTemplate/blob/main/MLB_project_4_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from openai import OpenAI#
import numpy as np#

In [9]:
client = OpenAI(api_key="")

In [10]:
with open('messi.txt', 'r', encoding = 'latin-1') as f:#
    text = f.read()
    print(text)

Lionel AndrÃ©s Messi[note 1] (Spanish pronunciation: [ljoËnel anËdÉ¾es Ëmesi] â; born 24 June 1987), also known as Leo Messi, is an Argentine professional footballer who plays as a forward for and captains both Major League Soccer club Inter Miami and the Argentina national team. Widely regarded as one of the greatest players of all time, Messi set numerous records for individual accolades won throughout his professional footballing career such as eight Ballon d'Or awards and eight times being named the world's best player by FIFA.[note 2] He is the most decorated player in the history of professional football having won 45 team trophies,[note 3] including twelve league titles, four UEFA Champions Leagues, two Copa AmÃ©ricas, and one FIFA World Cup. Messi holds the records for most European Golden Shoes (6), most goals for a single club (672, with Barcelona), most goals (474), hat-tricks (36) and assists (192) in La Liga, most matches played (39), assists (18) and goal contributio

In [11]:
CHUNNK_SIZE = 100#
chunks = []
current_place = 0
while current_place < len(text):
    current_chunk = text[current_place : current_place + CHUNNK_SIZE]
    chunks.append(current_chunk)
    current_place += CHUNNK_SIZE
print(chunks)

['Lionel AndrÃ©s Messi[note 1] (Spanish pronunciation: [ljoË\x88nel anË\x88dÉ¾es Ë\x88mesi] â\x93\x98; born 24 June 1', '987), also known as Leo Messi, is an Argentine professional footballer who plays as a forward for an', 'd captains both Major League Soccer club Inter Miami and the Argentina national team. Widely regarde', 'd as one of the greatest players of all time, Messi set numerous records for individual accolades wo', "n throughout his professional footballing career such as eight Ballon d'Or awards and eight times be", "ing named the world's best player by FIFA.[note 2] He is the most decorated player in the history of", ' professional football having won 45 team trophies,[note 3] including twelve league titles, four UEF', 'A Champions Leagues, two Copa AmÃ©ricas, and one FIFA World Cup. Messi holds the records for most Eu', 'ropean Golden Shoes (6), most goals for a single club (672, with Barcelona), most goals (474), hat-t', 'ricks (36) and assists (192) in La Liga, most m

In [12]:
embedded_chunks = []
for chunk in chunks:#
    response = client.embeddings.create(
        input=chunk,#
        model="text-embedding-3-small"
    )
    embedded_chunks.append( (chunk, response.data[0].embedding) )
print(embedded_chunks)

[('Lionel AndrÃ©s Messi[note 1] (Spanish pronunciation: [ljoË\x88nel anË\x88dÉ¾es Ë\x88mesi] â\x93\x98; born 24 June 1', [0.026616455987095833, -0.04776233807206154, -0.012677007354795933, -0.015748945996165276, -0.0011572372168302536, 0.001234167255461216, 0.06164918467402458, 0.008353148587048054, -0.02743704244494438, 0.0012499477015808225, -0.05399037525057793, 0.00801123771816492, -0.011782779358327389, -0.01675889827311039, 0.023775964975357056, 0.04203927144408226, -0.06139669567346573, -0.003056158311665058, -0.020125407725572586, -0.039808958768844604, 0.060092173516750336, -0.005304880440235138, 0.002348665613681078, -0.010793867520987988, 0.009157954715192318, -0.007932335138320923, -0.022850174456834793, 0.0031429510563611984, -0.0066383336670696735, 0.010793867520987988, 0.06042882427573204, -0.016853582113981247, 0.006554170977324247, -0.006301682908087969, 0.01978875696659088, -0.005538958124816418, -0.025417137891054153, -0.010062703862786293, -0.01027837023139, -0.0208

In [13]:
def cosine_simularity(A, B):
    return np.dot(A,B) / ( np.linalg.norm(A) * np.linalg.norm(B) ) # ABcos/A/|A|

In [14]:
question = "where was messi born?"

In [15]:
question_embedding = client.embeddings.create(
        input=question,
        model="text-embedding-3-small"
    ).data[0].embedding
print(question_embedding)

[-0.01681765913963318, -0.06454616039991379, -0.03615947067737579, 0.004479867871850729, -0.0291880052536726, -0.061461083590984344, 0.036299701780080795, 0.01389284897595644, -0.045835383236408234, -0.04118774086236954, -0.07047591358423233, 0.007121712900698185, -0.006340428255498409, -0.0232181865721941, -0.00910998322069645, 0.05244625732302666, -0.023418515920639038, 0.02794596180319786, -0.011559011414647102, -0.027645468711853027, 0.022637231275439262, -0.004958154633641243, -0.012159999459981918, 0.005383854731917381, 0.02494101971387863, -0.005659307818859816, 0.014133244752883911, -0.01681765913963318, 0.006240263115614653, 0.030550245195627213, 0.04555492475628853, -0.03167209029197693, 0.0325535386800766, -0.017278417944908142, 0.010477231815457344, -0.003931466024369001, 0.01618662104010582, -0.023839207366108894, -0.02570227161049843, -0.00020862437668256462, -0.057414427399635315, -0.040366388857364655, -0.0018192421412095428, 0.029428400099277496, -0.01380270067602396, 

In [16]:
best_cosine_similarity = 0
index = 0
for i in range(len(embedded_chunks)):
    similarity = cosine_simularity(question_embedding, embedded_chunks[i][1])
    if similarity > best_cosine_similarity: # checking the index for the highest comparison.
        best_cosine_similarity = similarity
        index = i

In [17]:
print(f"Relevant information: {embedded_chunks[index][0]}")

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user", "content": f"User question: {question}. Potentially useful information: {embedded_chunks[index][0]}"# relevant information.
        }
    ]
)

Relevant information:  born on 24 June 1987 in Rosario, Santa Fe Province,[21] the third of four children of Jorge Messi, 


In [18]:
print(completion.choices[0].message.content)#semantic search

Lionel Messi was born on 24 June 1987 in Rosario, Santa Fe Province, Argentina.
